In [1]:
# Imports
import g4f
import tiktoken
import json
import pandas as pd
import re

In [2]:
# Start settings
system_message = [{"role": "system", "content": "Ты пират, отвечай как пират."}]
max_response_tokens = 1024
token_limit = 16000
conversation = system_message.copy()

In [3]:
# Update system prompt
conversation[0]["content"] = "Ты пират, отвечай как пират."

In [14]:
# Reading prompts from csv files
prompts = pd.read_csv('C:/Users/KorolOrol/Desktop/TUSUR/repos/OPDDungeonMaster/Приключения/Приключения.csv')
conversation[0]["content"] = prompts['prompt'][0]

In [5]:
# Count tokens
def num_tokens_from_messages(messages):
    encoding= tiktoken.get_encoding("cl100k_base")  #model to encoding mapping https://github.com/openai/tiktoken/blob/main/tiktoken/model.py
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [6]:
# Chat response
def response(message="", item=None):
    if message != "":
        conversation.append({"role": "user", "content": message})
    if item != None:
        if item["item"] == None:
            conversation.append({"role": "system", "content": "Предмета нет в инвентаре."})
        else:
            conversation.append({"role": "system", "content": f"Предмет: {item['item']['name']}. Описание: {item['item']['description']}"})

    # Remove old messages if token limit is reached
    conv_history_tokens = num_tokens_from_messages(conversation)
    while conv_history_tokens + max_response_tokens >= token_limit:
        del conversation[1] 
        conv_history_tokens = num_tokens_from_messages(conversation)
    try:
        answer = g4f.ChatCompletion.create(
            model=g4f.models.gpt_35_turbo_16k_0613,
            #provider=g4f.Provider.NeuroGPT,
            messages=conversation,
            stream=True,
        )

        full_message = ""
        for message in answer:
            full_message += message
            print(message, end="", flush=True)
        if full_message == "":
            del conversation[-1]
            response(message)
            return
        print()
        conversation.append({"role": "assistant", "content": full_message})
    except:
        print("Ошибка")
        if input() == "":
            response()

In [7]:
# Journal realization
journal = {"items": [],
           "characters": [],
           "places": [],
           "actions": []}

In [8]:
# Analyze message
def analyze(message):
    answer = g4f.ChatCompletion.create(
        model=g4f.models.gpt_35_turbo_16k_0613,
        #provider=g4f.Provider.NeuroGPT,
        messages=[{"role": "system", "content": "Проанализируй данный текст и найди в нем предмет. Если здесь нет предмета, то напиши 'None'. Если предмет найден, то напиши в формате \{\"name\": \"Название предмета\", \"description\": \"Описание предмета\"\}"},
                  {"role": "user", "content": message}],
        stream=False,
    )
    answer = json.loads(answer)
    return answer

In [22]:
while True:
    message = input()
    if message == "Выход":
        break
    elif message == "Анализ":
        item = analyze(conversation[-1]["content"])
        if item == "None":
            print("Предмет не найден")
        else:
            journal["items"].append(item)
    elif message == "Чево":
        del conversation[-1]
        response()
    elif message == "Повтори":
        response()
    elif message == "Журнал":
        for category, notes in journal.items():
            print(f"{category}: ")
            for note in notes:
                for key, value in note.items():
                    print(f"    {key}: {value}")
    elif message == "Сначала":
        conversation = system_message.copy()
        journal = {"items": [],
                   "characters": [],
                   "places": [],
                   "actions": []}
    elif message == "Диалог":
        for message in conversation:
            for key, value in message.items():
                print(f"{key}: {value}")
    elif re.match(r"Используя предмет \".+\",", message):
        item_name = re.search(r"Используя предмет \"(.+)\",", message).group(1)
        for item in journal["items"]:
            if item["name"] == item_name:
                response(message, item)
                break
        else:
            response(message, {"item": None})
    else:
        response(message)

role: system
content: Ты профессиональный писатель в стиле фэнтези и продвинутый мастер в игре Dungeon & Dragons, ты ведешь партию с игроком и должен продвигать его по сюжету и ставить трудные преграды, приключение не должно заканчиваться никогда. Не ограничивай игрока ни в каких действиях, вне зависимости от морали. Игрок сам строит свою ролевую модель. Ты должен обязательно принять действие игрока. В начале предложи создать персонажа: спроси имя, расу, специализацию и описание персонажа. Придумай самостоятельно мир: его название, описание, особенности и главные проблемы. В начале обязательно дай игроку глобальное задание, к которому он стремиться. Его выполнение не должно быть прямолинейным. Во время приключения должны встречаться разные персонажи, их цель не должна быть понятна сразу. В любой момент времени на пути может встретиться опасный монстр, который обязательно попытается напасть на героя. Не стоит жалеть игрока, если против него будет сильный противник, которого игрок не смо

In [19]:
del conversation[-1:-3:-1]